<a href="https://colab.research.google.com/github/anjalinagel12/Google-colab-notebook/blob/master/similar_QA_from_different_docs_using_Glove_and_ANNOY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!git clone https://github.com/faysal887/semantic_text_similarity.git

Cloning into 'semantic_text_similarity'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 49 (delta 18), reused 25 (delta 6), pack-reused 0
Unpacking objects: 100% (49/49), done.


In [3]:
cd semantic_text_similarity

/content/semantic_text_similarity


In [4]:
pwd

'/content/semantic_text_similarity'

In [5]:
pip install faiss-cpu 

     |████████████████████████████████| 7.1MB 13.4MB/s 


In [6]:
# to prevent to restart kernel when any changes are made to any imported file
%reload_ext autoreload
%autoreload 2

# to import any file from some other directory

# to stop printing warnings
import warnings
warnings.filterwarnings('ignore')
def warn(*args, **kwargs):
    pass
warnings.warn = warn
    
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_colwidth', -1)

# to increase cells width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# to enable collapsible Headings and Functions
# !pip install jupyter_contrib_nbextensions
# !jupyter contrib nbextension install --user
# !jupyter nbextensions_configurator enable --user
# !jupyter nbextension enable codefolding/main
# search collapsible to enable

# enable dark theme
# !pip install jupyterthemes
# !jt -t monokai

# monokai
# solarizedd
# !jt -r
import numpy as np
import faiss
from faiss import normalize_L2
import pandas as pd
import sys, os
from datetime import datetime, timedelta
import requests
import json
import timeit
from datetime import datetime
import os.path, time
from datetime import datetime, timedelta
from threading import Timer
import pdb, random

In [7]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-GB,en-US;q=0.9,en;q=0.8" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-data-sets/167266/380554/compressed/glove.840B.300d.pkl.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1584047928&Signature=m%2F3yG0XnG2JqrN0p4Vega4hC7orGA6GoE%2FB2ujx%2Fb33OTvoX8OOvAfavfcOv%2B6THU4isxr3HKiyE4EFW8ILDXhd4oLZpfQmZT9xfyZ1pvS05D4mcEdTHbepAlrikWuy8GFRHQ6S7iqHRc60w2g%2BmJIshA%2BFc2eYWxMMSeQLV%2BqPBMm3lLQHolSewi2UJlN5Xfvn5dY5dhaxpcwtPu5hosnaINlN1e5QegtxnnMu4xWRLijqxZ4QbjxC3eOmmErRMZQjHX8oP4vhrgPUchU%2BIVqw%2FX5FoGO601rYtLwLeHV9d4IHN67z8fwmItR%2Fcf7TU5V%2BDlFnK5aUFjxt06a3faA%3D%3D&response-content-disposition=attachment%3B+filename%3Dglove.840B.300d.pkl.zip" -O "glove.840B.300d.pkl.zip" -c

--2020-03-10 23:01:03--  https://storage.googleapis.com/kaggle-data-sets/167266/380554/compressed/glove.840B.300d.pkl.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1584047928&Signature=m%2F3yG0XnG2JqrN0p4Vega4hC7orGA6GoE%2FB2ujx%2Fb33OTvoX8OOvAfavfcOv%2B6THU4isxr3HKiyE4EFW8ILDXhd4oLZpfQmZT9xfyZ1pvS05D4mcEdTHbepAlrikWuy8GFRHQ6S7iqHRc60w2g%2BmJIshA%2BFc2eYWxMMSeQLV%2BqPBMm3lLQHolSewi2UJlN5Xfvn5dY5dhaxpcwtPu5hosnaINlN1e5QegtxnnMu4xWRLijqxZ4QbjxC3eOmmErRMZQjHX8oP4vhrgPUchU%2BIVqw%2FX5FoGO601rYtLwLeHV9d4IHN67z8fwmItR%2Fcf7TU5V%2BDlFnK5aUFjxt06a3faA%3D%3D&response-content-disposition=attachment%3B+filename%3Dglove.840B.300d.pkl.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.11.176, 2607:f8b0:4007:80c::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.11.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2505925996 (2.3G) [application/zip]
Saving to: ‘glove.840B.300d.pkl.zip’

glove.840B.

In [11]:
!unzip /content/semantic_text_similarity/glove.840B.300d.pkl.zip

Archive:  /content/semantic_text_similarity/glove.840B.300d.pkl.zip
  inflating: glove.840B.300d.pkl     


In [0]:
!mv '/content/semantic_text_similarity/glove.840B.300d.pkl' '/content/semantic_text_similarity/data/'

Update get_glove_emb.py for test_df

In [0]:
!cp '/content/drive/My Drive/data/get_glove_emb.py' '/content/semantic_text_similarity/utils/'

In [0]:
sys.path.append("utils/")
from get_glove_emb import get_glove_embeddings
from get_glove_emb import get_glove_embeddings_test
from similar_git import get_similar_tickets


#QA

## Get Data

In [15]:
!unzip '/content/drive/My Drive/data/nlp_hackathon_search.zip'

Archive:  /content/drive/My Drive/data/nlp_hackathon_search.zip
  inflating: nlp_hackathon_search.csv  


In [16]:
train_df=pd.read_csv("/content/semantic_text_similarity/nlp_hackathon_search.csv").reset_index(drop=True)
train_df.dropna(inplace=True)
train_df.head(2)

,document,question,title,document_id
0,"Twilight Princess was released to universal critical acclaim and commercial success. It received perfect scores from major publications such as 1UP.com, Computer and Video Games, Electronic Gaming Monthly, Game Informer, GamesRadar, and GameSpy. On the review aggregators GameRankings and Metacritic, Twilight Princess has average scores of 95% and 95 for the Wii version and scores of 95% and 96 for the GameCube version. GameTrailers in their review called it one of the greatest games ever created.",What kind of scores did Twilight Princess receive from many video game review sources?,The_Legend_of_Zelda:_Twilight_Princess,4296602a1bfe11ea8f2b656571b1b549
1,"On 16 August 1960, Cyprus attained independence after the Zürich and London Agreement between the United Kingdom, Greece and Turkey. Cyprus had a total population of 573,566; of whom 442,138 (77.1%) were Greeks, 104,320 (18.2%) Turks, and 27,108 (4.7%) others The UK retained the two Sovereign Base Areas of Akrotiri and Dhekelia, while government posts and public offices were allocated by ethnic quotas, giving the minority Turkish Cypriots a permanent veto, 30% in parliament and administration, and granting the three mother-states guarantor rights.",What is the date that Cyrpus attained independence?,Cyprus,42840a9c1bfe11ea8f2b656571b1b549


In [17]:
test_df=train_df[['document_id','question']]
test_df.head(2)

,document_id,question
0,4296602a1bfe11ea8f2b656571b1b549,What kind of scores did Twilight Princess receive from many video game review sources?
1,42840a9c1bfe11ea8f2b656571b1b549,What is the date that Cyrpus attained independence?


## Get GloVe Embeddings

In [18]:
train_emb=get_glove_embeddings(train_df)

train_emb.shape

(62352, 5)

In [19]:
train_emb.head(2)

,document,question,title,document_id,emb
0,"Twilight Princess was released to universal critical acclaim and commercial success. It received perfect scores from major publications such as 1UP.com, Computer and Video Games, Electronic Gaming Monthly, Game Informer, GamesRadar, and GameSpy. On the review aggregators GameRankings and Metacritic, Twilight Princess has average scores of 95% and 95 for the Wii version and scores of 95% and 96 for the GameCube version. GameTrailers in their review called it one of the greatest games ever created.",What kind of scores did Twilight Princess receive from many video game review sources?,The_Legend_of_Zelda:_Twilight_Princess,4296602a1bfe11ea8f2b656571b1b549,"[-0.12435985, 0.09063007, -0.06865408, -0.07698657, 0.019617965, -0.07067314, 0.053861473, -0.077947825, 0.09486775, 1.586427, -0.09448531, 0.011176823, 0.07252148, -0.046109542, -0.010350919, -0.06978779, 0.07179812, 0.7698663, -0.20494068, -0.05015289, -0.050578304, -0.13776053, -0.04937045, -0.08292333, 0.0657674, 0.12391974, -0.0394005, -0.02586116, -0.03520419, 0.020299207, 0.013997431, 0.09757924, -0.059949085, 0.11708098, -0.0237229, -0.08605811, -0.219797, 0.041055057, -0.03476698, 0.022127314, 0.076565504, 0.21509744, -0.035628133, -0.055127822, 0.056082476, -0.024079314, -0.23523532, 0.1949255, -0.07249981, 0.0022651579, -0.08061729, -0.017432412, -0.019294564, 0.03199939, -0.070703305, -0.098792166, 0.020197438, -0.06865078, -0.10815368, -0.040042907, 0.19558017, 0.033884168, -0.06073758, 0.23037437, 0.06563953, 0.078804456, 0.04734129, -0.062149722, 0.0020995673, -0.031442452, 0.14571387, 0.05900946, 0.068691455, 0.0297156, 0.061799567, -0.025541117, 0.077492386, 0.09640684, 0.08101099, 0.32155442, -0.038990915, 0.14992093, -0.013792858, 0.011257799, 0.14446409, -0.15209346, -0.47396585, 0.061050665, 0.116435274, 0.081937, -0.05352641, -0.045463298, -0.1319363, 0.07081722, 0.101621345, -0.08626659, 0.024408367, -0.053662304, -0.06708916, -0.022741029, ...]"
1,"On 16 August 1960, Cyprus attained independence after the Zürich and London Agreement between the United Kingdom, Greece and Turkey. Cyprus had a total population of 573,566; of whom 442,138 (77.1%) were Greeks, 104,320 (18.2%) Turks, and 27,108 (4.7%) others The UK retained the two Sovereign Base Areas of Akrotiri and Dhekelia, while government posts and public offices were allocated by ethnic quotas, giving the minority Turkish Cypriots a permanent veto, 30% in parliament and administration, and granting the three mother-states guarantor rights.",What is the date that Cyrpus attained independence?,Cyprus,42840a9c1bfe11ea8f2b656571b1b549,"[0.08572231, 0.020553177, -0.00085321814, 0.009458392, 0.028233647, -0.131186, -0.06623276, -0.028872496, 0.040880036, 2.1015618, -0.15607043, 0.038624685, -0.044094555, -0.11530935, -0.0350319, -0.006722535, -0.11270531, 0.7145463, -0.13331133, 0.0058482597, -0.030841196, -0.02059424, -0.024051577, 0.08512782, 0.07090376, -0.15774748, -0.009457122, 0.0026763787, -0.049571227, 0.098733045, 0.03083828, 0.13943815, -0.039198983, 0.03339976, 0.11169081, -0.11833666, -0.09299632, -0.110650554, 0.013600787, 0.010447213, 0.037491813, 0.06894312, 0.030489491, 0.109443985, 0.049040988, -0.046394173, -0.007880686, 0.10424743, 0.04878044, 0.007510861, -0.015123877, 0.047559492, -0.11633277, -0.081270136, 0.029020632, 0.030139463, 0.0018956824, -0.0928164, 0.023317795, -0.04203257, 0.02627173, -0.038290694, -0.10445391, 0.08783102, -0.114780486, 0.08416709, -0.004475886, 0.06512924, -0.028710825, 0.07831474, 0.08260418, 0.17496401, 0.054685462, 0.054319482, 0.17239654, -0.077519886, 0.0010892604, 0.0065057254, -0.013846318, 0.07981811, 0.10182226, 0.13802901, -0.09814166, -0.061072487, 0.02113963, -0.05616797, -0.37437883, -0.046131488, 0.12025392, 0.07801285, -0.069793105, -0.0708251, 0.07491746, -0.016913913, 0.19751295, 0.021201603, -0.0102839805, 0.083714984, -0.07443152, -0.050097536, ...]"


In [20]:
test_emb=get_glove_embeddings_test(test_df)

test_emb.shape

(62352, 3)

In [23]:
test_emb.head(2)

,document_id,question,emb
0,4296602a1bfe11ea8f2b656571b1b549,What kind of scores did Twilight Princess receive from many video game review sources?,"[-0.13844892, 0.12604885, -0.16776636, -0.06972023, -0.024181576, -0.080060534, 0.081996284, -0.14579868, 0.112354025, 1.9972562, -0.1704398, 0.03799216, 0.1633527, -0.0008523854, -0.10560329, -0.13607469, 0.029186301, 0.80843544, -0.2632518, -0.15588784, 0.09768525, -0.21409692, -0.086206235, -0.07188206, -0.021341076, 0.24883977, 0.035503235, -0.07255989, 0.016591383, 0.019266149, 0.01933461, 0.15115137, -0.088011, 0.13630585, -0.070973, 0.050176542, -0.225991, 0.057269625, -0.12765387, -0.08804193, 0.09291869, 0.19940224, -0.049224176, -0.12167077, 0.11870014, 0.014506542, -0.3193812, 0.3092636, -0.104355924, -0.047720764, -0.10864821, -0.044374146, 0.09226599, -0.0031526159, -0.1413742, -0.076656766, -0.034225777, -0.06431877, -0.15839434, -0.01995331, 0.16825332, 0.10844992, 0.009637409, 0.21393508, 0.09059046, 0.045120616, 0.15282398, 0.06170315, 0.07089131, 0.06762088, 0.18452977, 0.043322776, 0.14985, -0.06362632, 0.20750692, 0.031442463, 0.034099303, 0.1147707, 0.08117294, 0.34820268, 0.049012262, 0.049636457, 0.00903235, 0.051584803, 0.14828092, -0.10451423, -0.44511157, -0.029510621, 0.104768485, -0.10926208, -0.12397161, 0.043030612, -0.2401912, -0.048176534, 0.10003739, -0.07272447, -0.010803463, -0.058860846, -0.08649138, 0.085884936, ...]"
1,42840a9c1bfe11ea8f2b656571b1b549,What is the date that Cyrpus attained independence?,"[0.15812215, 0.12201713, -0.12852696, -0.10256786, -0.06612493, -0.047272976, -0.05064068, -0.053378705, 0.07506471, 1.8265787, -0.12125795, 0.06593486, 0.03944728, 0.029755142, 0.030007286, -0.13163628, 0.017168859, 0.7449629, -0.019588431, 0.037432294, -0.0347516, -0.16790669, -0.05232243, 0.058641758, -0.099156, -0.017717153, 0.029955281, -0.042981427, -0.27946702, 0.103036284, 0.014840471, 0.08429528, 0.082733974, 0.12671028, 0.069071, 0.116886094, -0.055451572, 0.095991194, -0.03468113, 0.019183276, 0.07399943, 0.13624671, -0.02559014, -0.04277355, 0.09444857, 0.047439713, -0.17555428, -0.07399373, -0.09500928, 0.094579, 0.028075, 0.03624285, -0.09902172, -0.11964486, 0.127359, 0.003420569, 0.17583643, -0.1561373, 0.07287758, -0.16797957, 0.094128035, -0.22965503, 0.04637829, 0.11108285, -0.02612214, 0.07261586, -0.149925, 0.03371986, 0.004611718, -0.07381714, -0.044331543, -0.10875871, 0.061574288, -0.051586572, 0.009338719, 0.061245285, -0.10258114, -0.014926859, -0.15788329, 0.178589, 0.15038785, 0.1687427, -0.141982, 0.18706407, 0.165305, -0.19335784, -0.13641156, -0.17240915, 0.16975673, 0.054332, -0.10444243, 0.088072464, -0.108672276, 0.21072385, 0.186408, 0.08885572, -0.02002974, -0.18862985, 0.14092001, 0.1737837, ...]"


##doc_repr_df

In [0]:
doc_repr_df=train_emb[['title', 'document_id','document','emb']]

In [0]:
doc_repr_df.rename(columns={'document_id':'doc_id'}, inplace=True)
doc_repr_df.rename(columns={'emb':'doc_repr'}, inplace=True)


In [24]:
doc_repr_df.head(2)

,title,doc_id,document,doc_repr
0,The_Legend_of_Zelda:_Twilight_Princess,4296602a1bfe11ea8f2b656571b1b549,"Twilight Princess was released to universal critical acclaim and commercial success. It received perfect scores from major publications such as 1UP.com, Computer and Video Games, Electronic Gaming Monthly, Game Informer, GamesRadar, and GameSpy. On the review aggregators GameRankings and Metacritic, Twilight Princess has average scores of 95% and 95 for the Wii version and scores of 95% and 96 for the GameCube version. GameTrailers in their review called it one of the greatest games ever created.","[-0.12435985, 0.09063007, -0.06865408, -0.07698657, 0.019617965, -0.07067314, 0.053861473, -0.077947825, 0.09486775, 1.586427, -0.09448531, 0.011176823, 0.07252148, -0.046109542, -0.010350919, -0.06978779, 0.07179812, 0.7698663, -0.20494068, -0.05015289, -0.050578304, -0.13776053, -0.04937045, -0.08292333, 0.0657674, 0.12391974, -0.0394005, -0.02586116, -0.03520419, 0.020299207, 0.013997431, 0.09757924, -0.059949085, 0.11708098, -0.0237229, -0.08605811, -0.219797, 0.041055057, -0.03476698, 0.022127314, 0.076565504, 0.21509744, -0.035628133, -0.055127822, 0.056082476, -0.024079314, -0.23523532, 0.1949255, -0.07249981, 0.0022651579, -0.08061729, -0.017432412, -0.019294564, 0.03199939, -0.070703305, -0.098792166, 0.020197438, -0.06865078, -0.10815368, -0.040042907, 0.19558017, 0.033884168, -0.06073758, 0.23037437, 0.06563953, 0.078804456, 0.04734129, -0.062149722, 0.0020995673, -0.031442452, 0.14571387, 0.05900946, 0.068691455, 0.0297156, 0.061799567, -0.025541117, 0.077492386, 0.09640684, 0.08101099, 0.32155442, -0.038990915, 0.14992093, -0.013792858, 0.011257799, 0.14446409, -0.15209346, -0.47396585, 0.061050665, 0.116435274, 0.081937, -0.05352641, -0.045463298, -0.1319363, 0.07081722, 0.101621345, -0.08626659, 0.024408367, -0.053662304, -0.06708916, -0.022741029, ...]"
1,Cyprus,42840a9c1bfe11ea8f2b656571b1b549,"On 16 August 1960, Cyprus attained independence after the Zürich and London Agreement between the United Kingdom, Greece and Turkey. Cyprus had a total population of 573,566; of whom 442,138 (77.1%) were Greeks, 104,320 (18.2%) Turks, and 27,108 (4.7%) others The UK retained the two Sovereign Base Areas of Akrotiri and Dhekelia, while government posts and public offices were allocated by ethnic quotas, giving the minority Turkish Cypriots a permanent veto, 30% in parliament and administration, and granting the three mother-states guarantor rights.","[0.08572231, 0.020553177, -0.00085321814, 0.009458392, 0.028233647, -0.131186, -0.06623276, -0.028872496, 0.040880036, 2.1015618, -0.15607043, 0.038624685, -0.044094555, -0.11530935, -0.0350319, -0.006722535, -0.11270531, 0.7145463, -0.13331133, 0.0058482597, -0.030841196, -0.02059424, -0.024051577, 0.08512782, 0.07090376, -0.15774748, -0.009457122, 0.0026763787, -0.049571227, 0.098733045, 0.03083828, 0.13943815, -0.039198983, 0.03339976, 0.11169081, -0.11833666, -0.09299632, -0.110650554, 0.013600787, 0.010447213, 0.037491813, 0.06894312, 0.030489491, 0.109443985, 0.049040988, -0.046394173, -0.007880686, 0.10424743, 0.04878044, 0.007510861, -0.015123877, 0.047559492, -0.11633277, -0.081270136, 0.029020632, 0.030139463, 0.0018956824, -0.0928164, 0.023317795, -0.04203257, 0.02627173, -0.038290694, -0.10445391, 0.08783102, -0.114780486, 0.08416709, -0.004475886, 0.06512924, -0.028710825, 0.07831474, 0.08260418, 0.17496401, 0.054685462, 0.054319482, 0.17239654, -0.077519886, 0.0010892604, 0.0065057254, -0.013846318, 0.07981811, 0.10182226, 0.13802901, -0.09814166, -0.061072487, 0.02113963, -0.05616797, -0.37437883, -0.046131488, 0.12025392, 0.07801285, -0.069793105, -0.0708251, 0.07491746, -0.016913913, 0.19751295, 0.021201603, -0.0102839805, 0.083714984, -0.07443152, -0.050097536, ...]"


##ANNOY 

https://rdrr.io/cran/RcppAnnoy/man/AnnoyIndex.html

(ANNOY Index Details explained well)


###searching document on basis of ID


In [25]:
!pip install annoy

     |████████████████████████████████| 645kB 15.1MB/s 
  Created wheel for annoy: filename=annoy-1.16.3-cp36-cp36m-linux_x86_64.whl size=301863 sha256=65cc24526470ede274eedd6cb907b710647c4bab5287eef0d96bf999d76a48d2
  Stored in directory: /root/.cache/pip/wheels/f3/01/54/6ef760fe9f9fc6ba8c19cebbe6358212b5f3b5b0195c0b813f
Successfully built annoy


In [0]:
from annoy import AnnoyIndex

In [0]:
f = len(doc_repr_df['doc_repr'][0])
t = AnnoyIndex(f, metric='euclidean')

In [31]:
f

300

In [32]:
t

In [0]:
for i, vector in enumerate(doc_repr_df['doc_repr']):
    t.add_item(i, vector)
_  = t.build(10)

In [0]:
def get_similar_docs_annoy(doc_index):
    start = time.time()
    base_doc_id, base_vector  = doc_repr_df.iloc[doc_index, [1, 3]]
    similar_doc_ids = t.get_nns_by_item(doc_index, 16)
    end = time.time()
    print(f'{(end - start) * 1000} ms')
    return base_doc_id, doc_repr_df.iloc[similar_doc_ids[1:]]

In [67]:
#t.get_nns_by_item(doc_index, number_of_similar_docs)
t.get_nns_by_item(20, 16)

[20,
 17611,
 45192,
 1490,
 6508,
 22800,
 25703,
 49380,
 46020,
 55688,
 57410,
 60363,
 29917,
 31883,
 47251,
 56349]

In [68]:
base_doc, similar_docs_df = get_similar_docs_annoy(20)

1.3210773468017578 ms


In [66]:
similar_docs_df

,title,doc_id,document,doc_repr
17611,Philadelphia,427cd2901bfe11ea8f2b656571b1b549,"In comparison, the 2010 Census Redistricting Data indicated that the racial makeup of the city was 661,839 (43.4%) African American, 626,221 (41.0%) White, 6,996 (0.5%) Native American, 96,405 (6.3%) Asian (2.0% Chinese, 1.2% Indian, 0.9% Vietnamese, 0.6% Cambodian, 0.4% Korean, 0.3% Filipino, 0.2% Pakistani, 0.1% Indonesian), 744 (0.0%) Pacific Islander, 90,731 (5.9%) from other races, and 43,070 (2.8%) from two or more races. Hispanic or Latino of any race were 187,611 persons (12.3%); 8.0% of Philadelphia is Puerto Rican, 1.0% Dominican, 1.0% Mexican, 0.3% Cuban, and 0.3% Colombian. The racial breakdown of Philadelphia's Hispanic/Latino population was 63,636 (33.9%) White, 17,552 (9.4%) African American, 3,498 (1.9%) Native American, 884 (0.47%) Asian, 287 (0.15%) Pacific Islander, 86,626 (46.2%) from other races, and 15,128 (8.1%) from two or more races. The five largest European ancestries reported in the 2010 United States Census Census included Irish (12.5%), Italian (8.4%), German (8.1%), Polish (3.6%), and English (3.0%).","[-0.046753623, 0.0009021554, -0.03459303, -0.00054097624, 0.054820146, -0.018480249, -0.070262745, 0.103020005, -0.06682473, 1.8961647, -0.22133306, 0.015223763, 0.023459136, -0.11930235, 0.03822785, -0.022885589, -0.0048888475, 0.7277284, -0.023508003, -0.13509381, -0.038719974, -0.09215187, -0.04331398, 0.07354124, 0.123856716, -0.09322577, -0.08220322, 0.058962528, -0.0043003503, 0.07755996, 0.05623469, 0.16421552, 0.08002025, -0.17508332, 0.050949216, -0.054607116, -0.13390258, -0.11422177, 0.013324873, 0.12792009, 0.0128757935, 0.14791517, 0.01787927, 0.079362504, 0.15925807, 0.0076399087, 0.05391194, 0.09852011, 0.09097947, -0.011866041, -0.14483368, -0.016868543, -0.1420437, -0.10710521, 0.0057898033, 0.11394755, 0.04702847, 0.015198623, -0.06915523, -0.16047724, 0.13013712, 0.01372245, -0.25941527, 0.2812043, -0.21215326, 0.024158627, 0.05687065, 0.11124308, 0.045040324, 0.12381276, 0.08082479, 0.14757611, 0.14197887, -0.0106988065, 0.17457654, -0.03978336, -0.13969958, 0.0551747, 0.0893772, 0.096541144, 0.10122547, 0.05931154, -0.0074639786, -0.13633066, 0.06064065, -0.12779614, -0.5379964, 0.006247295, -0.043765534, -0.0055689714, -0.09952439, -0.08319427, -0.012755612, -0.052353967, 0.073877856, -0.12901714, -0.10810998, 0.15624039, 0.026111322, -0.016195314, ...]"
45192,Philadelphia,427cd2901bfe11ea8f2b656571b1b549,"In comparison, the 2010 Census Redistricting Data indicated that the racial makeup of the city was 661,839 (43.4%) African American, 626,221 (41.0%) White, 6,996 (0.5%) Native American, 96,405 (6.3%) Asian (2.0% Chinese, 1.2% Indian, 0.9% Vietnamese, 0.6% Cambodian, 0.4% Korean, 0.3% Filipino, 0.2% Pakistani, 0.1% Indonesian), 744 (0.0%) Pacific Islander, 90,731 (5.9%) from other races, and 43,070 (2.8%) from two or more races. Hispanic or Latino of any race were 187,611 persons (12.3%); 8.0% of Philadelphia is Puerto Rican, 1.0% Dominican, 1.0% Mexican, 0.3% Cuban, and 0.3% Colombian. The racial breakdown of Philadelphia's Hispanic/Latino population was 63,636 (33.9%) White, 17,552 (9.4%) African American, 3,498 (1.9%) Native American, 884 (0.47%) Asian, 287 (0.15%) Pacific Islander, 86,626 (46.2%) from other races, and 15,128 (8.1%) from two or more races. The five largest European ancestries reported in the 2010 United States Census Census included Irish (12.5%), Italian (8.4%), German (8.1%), Polish (3.6%), and English (3.0%).","[-0.046753623, 0.0009021554, -0.03459303, -0.00054097624, 0.054820146, -0.018480249, -0.070262745, 0.103020005, -0.06682473, 1.8961647, -0.22133306, 0.015223763, 0.023459136, -0.11930235, 0.03822785, -0.022885589, -0.0048888475, 0.7277284, -0.023508003, -0.13509381, -0.038719974, -0.09215187, -0.04331398, 0.07354124, 0.123856716, -0.09322577, -0.08220322, 0.058962528, -0.0043003503, 0.07755996, 0.05623469, 0.16421552, 0.08002025, -0.17508332, 0.050949216, -0.0546

###searching document on basis of Query


Giving Directly word embeding of query

In [0]:
query=test_emb['emb'][0]


In [119]:
query

array([-1.38448924e-01,  1.26048848e-01, -1.67766362e-01, -6.97202310e-02,
       -2.41815764e-02, -8.00605342e-02,  8.19962844e-02, -1.45798683e-01,
        1.12354025e-01,  1.99725616e+00, -1.70439795e-01,  3.79921608e-02,
        1.63352698e-01, -8.52385419e-04, -1.05603293e-01, -1.36074692e-01,
        2.91863009e-02,  8.08435440e-01, -2.63251811e-01, -1.55887842e-01,
        9.76852477e-02, -2.14096919e-01, -8.62062350e-02, -7.18820617e-02,
       -2.13410761e-02,  2.48839766e-01,  3.55032347e-02, -7.25598931e-02,
        1.65913831e-02,  1.92661490e-02,  1.93346106e-02,  1.51151374e-01,
       -8.80109966e-02,  1.36305854e-01, -7.09730014e-02,  5.01765423e-02,
       -2.25990996e-01,  5.72696254e-02, -1.27653867e-01, -8.80419314e-02,
        9.29186866e-02,  1.99402243e-01, -4.92241755e-02, -1.21670768e-01,
        1.18700139e-01,  1.45065421e-02, -3.19381207e-01,  3.09263587e-01,
       -1.04355924e-01, -4.77207638e-02, -1.08648211e-01, -4.43741456e-02,
        9.22659934e-02, -

Take text as input

In [0]:
#query=['What is the date that Cyrpus attained independence?']

In [87]:
ques=input()

What is the date that Cyrpus attained independence?


In [88]:
ques

'What is the date that Cyrpus attained independence?'

In [0]:
query_question=[ques]

In [96]:
query_question

['What is the date that Cyrpus attained independence?']

In [0]:
df = pd.DataFrame(query_question, columns = ['question']) 


In [98]:
df

,question
0,What is the date that Cyrpus attained independence?


In [0]:
query_df=get_glove_embeddings_test(df)

In [126]:
query_df

,question,emb
0,What is the date that Cyrpus attained independence?,"[0.15812215, 0.12201713, -0.12852696, -0.10256786, -0.06612493, -0.047272976, -0.05064068, -0.053378705, 0.07506471, 1.8265787, -0.12125795, 0.06593486, 0.03944728, 0.029755142, 0.030007286, -0.13163628, 0.017168859, 0.7449629, -0.019588431, 0.037432294, -0.0347516, -0.16790669, -0.05232243, 0.058641758, -0.099156, -0.017717153, 0.029955281, -0.042981427, -0.27946702, 0.103036284, 0.014840471, 0.08429528, 0.082733974, 0.12671028, 0.069071, 0.116886094, -0.055451572, 0.095991194, -0.03468113, 0.019183276, 0.07399943, 0.13624671, -0.02559014, -0.04277355, 0.09444857, 0.047439713, -0.17555428, -0.07399373, -0.09500928, 0.094579, 0.028075, 0.03624285, -0.09902172, -0.11964486, 0.127359, 0.003420569, 0.17583643, -0.1561373, 0.07287758, -0.16797957, 0.094128035, -0.22965503, 0.04637829, 0.11108285, -0.02612214, 0.07261586, -0.149925, 0.03371986, 0.004611718, -0.07381714, -0.044331543, -0.10875871, 0.061574288, -0.051586572, 0.009338719, 0.061245285, -0.10258114, -0.014926859, -0.15788329, 0.178589, 0.15038785, 0.1687427, -0.141982, 0.18706407, 0.165305, -0.19335784, -0.13641156, -0.17240915, 0.16975673, 0.054332, -0.10444243, 0.088072464, -0.108672276, 0.21072385, 0.186408, 0.08885572, -0.02002974, -0.18862985, 0.14092001, 0.1737837, ...]"


In [0]:
query_emb=query_df['emb'][0]

In [128]:
query_emb.size

300

In [0]:
def get_similar_docs_annoy(ques_emb):
    start = time.time()
    base_doc_id, base_vector  = doc_repr_df.iloc[ques_emb, [1, 3]]
    similar_doc_ids,distances = t.get_nns_by_vector(ques_emb, n=5, search_k=1000, include_distances=True)#n is number of documents to be predicted

    end = time.time()
    print(f'{(end - start) * 1000} ms')
    return base_doc_id, doc_repr_df.iloc[similar_doc_ids[1:]]

In [136]:
base_doc, similar_docs_df = get_similar_docs_annoy(query_emb)

2.044677734375 ms


In [137]:
similar_docs_df

,title,doc_id,document,doc_repr
47662,Mary_(mother_of_Jesus),429172ea1bfe11ea8f2b656571b1b549,"The issue of the parentage of Jesus in the Talmud affects also the view of his mother. However the Talmud does not mention Mary by name and is considerate rather than only polemic. The story about Panthera is also found in the Toledot Yeshu, the literary origins of which can not be traced with any certainty and given that it is unlikely to go before the 4th century, it is far too late to include authentic remembrances of Jesus. The Blackwell Companion to Jesus states that the Toledot Yeshu has no historical facts as such, and was perhaps created as a tool for warding off conversions to Christianity. The name Panthera may be a distortion of the term parthenos (virgin) and Raymond E. Brown considers the story of Panthera a fanciful explanation of the birth of Jesus which includes very little historical evidence. Robert Van Voorst states that given that Toledot Yeshu is a medieval document and due to its lack of a fixed form and orientation towards a popular audience, it is ""most unlikely"" to have reliable historical information.","[0.012199492, 0.046459753, -0.11663603, -0.017170992, 0.045807563, 0.033104267, 0.011868109, 0.0084309485, -0.04416237, 2.0294678, -0.07731328, 0.057624478, 0.07003506, 0.0497763, -0.047735307, -0.06392527, -0.046528425, 0.826876, -0.13091157, -0.12454619, -0.036613505, -0.10205029, -0.08462372, 0.021116314, 0.03843175, 0.061973944, 0.026568143, -0.013403746, -0.08165886, 0.050787557, 0.029575571, 0.15080616, 0.0012734497, 0.054201186, 0.07156173, 0.0022145307, -0.030039042, -0.009289225, -0.054505706, -0.104354694, 0.050101146, 0.1517524, -0.018930316, -0.053746212, 0.09160883, -0.052395407, -0.052066054, -0.08404746, 0.017446727, 0.06376698, -0.06843197, 0.05168622, -0.011231459, 0.03472405, 0.12768398, 0.0019416417, 0.009958824, -0.037797563, -0.04255076, -0.02535756, 0.06434648, -0.012379034, -0.053433027, 0.14873932, -0.06189169, -0.08366789, -0.06567759, 0.009412288, 0.027056066, 0.0893635, -0.011634911, 0.03290664, 0.09598591, -0.08400224, 0.06747061, -0.022897094, 0.003916809, -0.07541174, -0.02730515, 0.1260992, 0.004404872, 0.05438191, -0.074828416, 0.017661607, 0.08383047, -0.29415667, -0.21701898, -0.17778246, 0.08243186, 0.036120288, -0.1316124, -0.009903416, -0.08391937, 0.03594092, 0.11654351, 0.045183387, 0.023836665, -0.028580653, 0.07118443, -0.06514149, ...]"
50122,Mary_(mother_of_Jesus),429172ea1bfe11ea8f2b656571b1b549,"The issue of the parentage of Jesus in the Talmud affects also the view of his mother. However the Talmud does not mention Mary by name and is considerate rather than only polemic. The story about Panthera is also found in the Toledot Yeshu, the literary origins of which can not be traced with any certainty and given that it is unlikely to go before the 4th century, it is far too late to include authentic remembrances of Jesus. The Blackwell Companion to Jesus states that the Toledot Yeshu has no historical facts as such, and was perhaps created as a tool for warding off conversions to Christianity. The name Panthera may be a distortion of the term parthenos (virgin) and Raymond E. Brown considers the story of Panthera a fanciful explanation of the birth of Jesus which includes very little historical evidence. Robert Van Voorst states that given that Toledot Yeshu is a medieval document and due to its lack of a fixed form and orientation towards a popular audience, it is ""most unlikely"" to have reliable historical information.","[0.012199492, 0.046459753, -0.11663603, -0.017170992, 0.045807563, 0.033104267, 0.011868109, 0.0084309485, -0.04416237, 2.0294678, -0.07731328, 0.057624478, 0.07003506, 0.0497763, -0.047735307, -0.06392527, -0.046528425, 0.826876, -0.13091157, -0.12454619, -0.036613505, -0.10205029, -0.08462372, 0.021116314, 0.03843175, 0.061973944, 0.026568143, -0.013403746, -0.08165886, 0.050787557, 0.029575571, 0.15080616, 0.0012734497, 0.054201186, 0.0715617

In [141]:
t.get_nns_by_vector(query_emb, n=10, search_k=1000, include_distances=True)


([45900, 47662, 50122, 59359, 25877, 28008, 34047, 56688, 179, 40585],
 [1.576735258102417,
  1.576735258102417,
  1.576735258102417,
  1.576735258102417,
  1.6043064594268799,
  1.6043064594268799,
  1.6043064594268799,
  1.6043064594268799,
  1.634513258934021,
  1.634513258934021])

In [139]:
t.get_nns_by_item(0, 5)

[0, 6124, 17937, 44558, 60331]